Question 3

In [20]:
# Importing Libraries
import numpy as np
import matplotlib.pyplot as plt


# Creating Class Robot and defining functions
class Robot():
  def __init__(self, numberoflinks, DH_matrix,joint_types=-1):
    self.numberoflinks = numberoflinks
    self.DH_matrix = DH_matrix
    if(joint_types == -1):
      self.joint_types = [1]*numberoflinks
    else:
      self.joint_types = joint_types

  def transform_matrix(self):
    self.A = []
    self.R = []
    for i in range(self.numberoflinks):
      self.A.append(np.array([[np.cos(self.DH_matrix[i][0]), -np.sin(self.DH_matrix[i][0]) * np.cos(self.DH_matrix[i][3]), np.sin(self.DH_matrix[i][0]) * np.sin(self.DH_matrix[i][3]), self.DH_matrix[i][2] * np.cos(self.DH_matrix[i][0])],[np.sin(self.DH_matrix[i][0]), np.cos(self.DH_matrix[i][0]) * np.cos(self.DH_matrix[i][3]), -np.cos(self.DH_matrix[i][0]) * np.sin(self.DH_matrix[i][3]), self.DH_matrix[i][2] * np.sin(self.DH_matrix[i][0])],[0, np.sin(self.DH_matrix[i][3]), np.cos(self.DH_matrix[i][3]), self.DH_matrix[i][1]],[0, 0, 0, 1]])) 
      self.R.append(np.array([[np.cos(self.DH_matrix[i][0]), -np.sin(self.DH_matrix[i][0]) * np.cos(self.DH_matrix[i][3]), np.sin(self.DH_matrix[i][0]) * np.sin(self.DH_matrix[i][3])],[np.sin(self.DH_matrix[i][0]), np.cos(self.DH_matrix[i][0]) * np.cos(self.DH_matrix[i][3]), -np.cos(self.DH_matrix[i][0]) * np.sin(self.DH_matrix[i][3])],[0, np.sin(self.DH_matrix[i][3]), np.cos(self.DH_matrix[i][3])]]))
    self.A = np.array(self.A) # Array of Homogenous Transformation Matrices
    self.R = np.array(self.R) # Array of Rotation matrices
    
    # Creating array of R_0_n matrices (R01, R02, R03, R04 ...)
    self.R_0_n = []
    A = np.identity(3)
    for i in range(self.numberoflinks):
      self.R_0_n.append(A)
      A = np.matmul(A, self.R[i])
    self.R_0_n.append(A)
    self.R_0_n = np.array(self.R_0_n)

    # Creating array of H_0_n matrices (H01, H02, H03, H04 ...)
    self.H_0_n = []
    B = np.identity(4)
    for i in range(self.numberoflinks):
      self.H_0_n.append(B)
      B = np.matmul(B, self.A[i])
    self.H_0_n.append(B)
    self.H_0_n = np.array(self.H_0_n)
    
    # Calculating Final Transfornation Matrix
    T = np.asmatrix(np.identity(4))
    for j in range(self.numberoflinks):
        T = T * np.asmatrix(self.A[j]) #,dtype = 'int' for integer output
    self.T = T
    # print ("A Matrices are: \n", self.A)
    print ("Final Transformation Matrix: \n",T)
    # print ("Rotation Matrices are: \n", self.R)
    # print ("Rotation Matrices w.r.t base frame are: \n", self.R_0_n)
  
  def manipulator_jacobian(self):
    # Creating z terms
    self.z = []
    k = np.array([[0],[0],[1]])
    for i in range(self.numberoflinks+1):
      self.z.append(np.matmul(self.R_0_n[i],k))
    self.z = np.array(self.z)

    # Creating O terms
    d = np.array([[0],[0],[0],[1]])
    self.O = np.matmul(self.H_0_n,d)[:self.numberoflinks+1]
    self.On = np.delete(self.O,(3),axis=1)

    # Creating Jacobian Manipulator (Here J1 will be 0th element of array J and so on)
    self.J = []
    for i in range(self.numberoflinks):
      if self.joint_types[i] == 1:
        X = self.On[-1] - self.On[i]
        J_v = np.cross(self.z[i], X,axis = 0)
        J_v = np.vstack((J_v, self.z[i]))
        self.J.append(J_v)
      else:
        J_v = np.vstack((self.z[i], np.array([[0],[0],[0]])))
        self.J.append(J_v)
    self.J = np.array(self.J)
    self.J_v = np.delete(self.J, (3,4,5), axis = 1)
    print ("Manipulator Jacobian: \n", self.J)
    print ("End effector position: \n", self.On[-1])
    print ("End effector velocity: \n", "v = ", self.J_v, " x q_dot")

# Initializing object "robot" and calculating result
# Mention the number of links in the first link
# Enter the DH table row wise in the order: [THETA, D, A, ALPHA]
# Enter Rho values for each link: 1 for Revolute and 0 for Prismatic
robot = Robot(3,[[np.pi/3, 0, 100, 0],[np.pi/6, 0, 100, 0],[np.pi/3, 0, 100, 0]],joint_types = [1,0,1])
robot.transform_matrix()
robot.manipulator_jacobian()

Final Transformation Matrix: 
 [[ -0.8660254   -0.5          0.         -36.60254038]
 [  0.5         -0.8660254    0.         236.60254038]
 [  0.           0.           1.           0.        ]
 [  0.           0.           0.           1.        ]]
Manipulator Jacobian: 
 [[[-236.60254038]
  [ -36.60254038]
  [   0.        ]
  [   0.        ]
  [   0.        ]
  [   1.        ]]

 [[   0.        ]
  [   0.        ]
  [   1.        ]
  [   0.        ]
  [   0.        ]
  [   0.        ]]

 [[ -50.        ]
  [ -86.60254038]
  [   0.        ]
  [   0.        ]
  [   0.        ]
  [   1.        ]]]
End effector position: 
 [[-36.60254038]
 [236.60254038]
 [  0.        ]]
End effector velocity: 
 v =  [[[-236.60254038]
  [ -36.60254038]
  [   0.        ]]

 [[   0.        ]
  [   0.        ]
  [   1.        ]]

 [[ -50.        ]
  [ -86.60254038]
  [   0.        ]]]  x q_dot


Question 4

In [9]:
# Importing Libraries
import numpy as np
import matplotlib.pyplot as plt


# Creating Class Robot and defining functions
class Robot():
  def __init__(self, numberoflinks, DH_matrix,joint_types=-1):
    self.numberoflinks = numberoflinks
    self.DH_matrix = DH_matrix
    if(joint_types == -1):
      self.joint_types = [1]*numberoflinks
    else:
      self.joint_types = joint_types

  def transform_matrix(self):
    self.A = []
    self.R = []
    for i in range(self.numberoflinks):
      self.A.append(np.array([[np.cos(self.DH_matrix[i][0]), -np.sin(self.DH_matrix[i][0]) * np.cos(self.DH_matrix[i][3]), np.sin(self.DH_matrix[i][0]) * np.sin(self.DH_matrix[i][3]), self.DH_matrix[i][2] * np.cos(self.DH_matrix[i][0])],[np.sin(self.DH_matrix[i][0]), np.cos(self.DH_matrix[i][0]) * np.cos(self.DH_matrix[i][3]), -np.cos(self.DH_matrix[i][0]) * np.sin(self.DH_matrix[i][3]), self.DH_matrix[i][2] * np.sin(self.DH_matrix[i][0])],[0, np.sin(self.DH_matrix[i][3]), np.cos(self.DH_matrix[i][3]), self.DH_matrix[i][1]],[0, 0, 0, 1]])) 
      self.R.append(np.array([[np.cos(self.DH_matrix[i][0]), -np.sin(self.DH_matrix[i][0]) * np.cos(self.DH_matrix[i][3]), np.sin(self.DH_matrix[i][0]) * np.sin(self.DH_matrix[i][3])],[np.sin(self.DH_matrix[i][0]), np.cos(self.DH_matrix[i][0]) * np.cos(self.DH_matrix[i][3]), -np.cos(self.DH_matrix[i][0]) * np.sin(self.DH_matrix[i][3])],[0, np.sin(self.DH_matrix[i][3]), np.cos(self.DH_matrix[i][3])]]))
    self.A = np.array(self.A) # Array of Homogenous Transformation Matrices
    self.R = np.array(self.R) # Array of Rotation matrices
    
    # Creating array of R_0_n matrices (R01, R02, R03, R04 ...)
    self.R_0_n = []
    A = np.identity(3)
    for i in range(self.numberoflinks):
      self.R_0_n.append(A)
      A = np.matmul(A, self.R[i])
    self.R_0_n.append(A)
    self.R_0_n = np.array(self.R_0_n)

    # Creating array of H_0_n matrices (H01, H02, H03, H04 ...)
    self.H_0_n = []
    B = np.identity(4)
    for i in range(self.numberoflinks):
      self.H_0_n.append(B)
      B = np.matmul(B, self.A[i])
    self.H_0_n.append(B)
    self.H_0_n = np.array(self.H_0_n)
    
    # Calculating Final Transfornation Matrix
    T = np.asmatrix(np.identity(4))
    for j in range(self.numberoflinks):
        T = T * np.asmatrix(self.A[j]) #,dtype = 'int' for integer output
    self.T = T
    # print ("A Matrices are: \n", self.A)
    print ("Final Transformation Matrix: \n",T)
    # print ("Rotation Matrices are: \n", self.R)
    # print ("Rotation Matrices w.r.t base frame are: \n", self.R_0_n)
  
  def manipulator_jacobian(self):
    # Creating z terms
    self.z = []
    k = np.array([[0],[0],[1]])
    for i in range(self.numberoflinks+1):
      self.z.append(np.matmul(self.R_0_n[i],k))
    self.z = np.array(self.z)

    # Creating O terms
    d = np.array([[0],[0],[0],[1]])
    self.O = np.matmul(self.H_0_n,d)[:self.numberoflinks+1]
    self.On = np.delete(self.O,(3),axis=1)

    # Creating Jacobian Manipulator (Here J1 will be 0th element of array J and so on)
    self.J = []
    for i in range(self.numberoflinks):
      if self.joint_types[i] == 1:
        X = self.On[-1] - self.On[i]
        J_v = np.cross(self.z[i], X,axis = 0)
        J_v = np.vstack((J_v, self.z[i]))
        self.J.append(J_v)
      else:
        J_v = np.vstack((self.z[i], np.array([[0],[0],[0]])))
        self.J.append(J_v)
    self.J = np.array(self.J)
    self.J_v = np.delete(self.J, (3,4,5), axis = 1)
    print ("Manipulator Jacobian: \n", self.J)
    print ("End effector position: \n", self.On[-1])
    print ("End effector velocity: \n", "v = ", self.J_v, " x q_dot")

# Initializing object "robot" and calculating result
# Mention the number of links in the first link
# Enter the DH table row wise in the order: [THETA, D, A, ALPHA]
# Enter Rho values for each link: 1 for Revolute and 0 for Prismatic
print("SCARA Robot (RRP) \n")
SCARA = Robot(3,[[np.pi/6, 0, 50, 0],[np.pi/6, 0, 50, np.pi],[0, 50, 0, 0]],joint_types = [1,1,0])
SCARA.transform_matrix()
SCARA.manipulator_jacobian()

print("------------------------------------------------------------------- \n")

STANFORD = Robot(3,[[np.pi/6, 0, 0, -np.pi/2],[np.pi/6, 50, 0, np.pi/2],[0, 50, 0, 0]],joint_types = [1,1,0])
print("Stanford Manipulator (RRP) \n")
STANFORD.transform_matrix()
STANFORD.manipulator_jacobian()

SCARA Robot (RRP) 

Final Transformation Matrix: 
 [[ 5.00000000e-01  8.66025404e-01  1.06057524e-16  6.83012702e+01]
 [ 8.66025404e-01 -5.00000000e-01 -6.12323400e-17  6.83012702e+01]
 [ 0.00000000e+00  1.22464680e-16 -1.00000000e+00 -5.00000000e+01]
 [ 0.00000000e+00  0.00000000e+00  0.00000000e+00  1.00000000e+00]]
Manipulator Jacobian: 
 [[[-6.83012702e+01]
  [ 6.83012702e+01]
  [ 0.00000000e+00]
  [ 0.00000000e+00]
  [ 0.00000000e+00]
  [ 1.00000000e+00]]

 [[-4.33012702e+01]
  [ 2.50000000e+01]
  [ 0.00000000e+00]
  [ 0.00000000e+00]
  [ 0.00000000e+00]
  [ 1.00000000e+00]]

 [[ 1.06057524e-16]
  [-6.12323400e-17]
  [-1.00000000e+00]
  [ 0.00000000e+00]
  [ 0.00000000e+00]
  [ 0.00000000e+00]]]
End effector position: 
 [[ 68.30127019]
 [ 68.30127019]
 [-50.        ]]
End effector velocity: 
 v =  [[[-6.83012702e+01]
  [ 6.83012702e+01]
  [ 0.00000000e+00]]

 [[-4.33012702e+01]
  [ 2.50000000e+01]
  [ 0.00000000e+00]]

 [[ 1.06057524e-16]
  [-6.12323400e-17]
  [-1.00000000e+00]]] 

Question 11

In [8]:
# Importing Libraries
import numpy as np
import sympy as sp
import matplotlib.pyplot as plt

# Creating Class Robot and defining functions
class Robot():
  def __init__(self, numberoflinks, D_q, V_q):
    self.numberoflinks = numberoflinks
    self.D_q = D_q
    self.V_q = V_q

  def derive_equations(self):
    Phi = [0] * self.numberoflinks # Creating phi matrix
    c = [[[0] * self.numberoflinks] * self.numberoflinks] * self.numberoflinks # Creating c matrix
    T = [0] * self.numberoflinks # Matrix to store torque values for each link
    d = 0
    ct = 0

    # Calculating Christoffer Symbols
    for k in range(self.numberoflinks):
      for j in range(self.numberoflinks):
        for i in range(self.numberoflinks):
          c[i][j][k] = 0.5 * (sp.diff(self.D_q[k][j], q[i]) + sp.diff(self.D_q[k][i], q[j]) - sp.diff(self.D_q[i][j], q[k]))
    
    for k in range(self.numberoflinks): #corresponds to each link
      Phi[k] = sp.diff(self.V_q, q[k])
      for j in range(self.numberoflinks):
        d = d + self.D_q[k][j] * qddot[j] 
        for i in range(self.numberoflinks):
          ct = ct + c[i][j][k] * qdot[i] * qdot[j]
      T[k] = d + ct + Phi[k]

    for i in range(self.numberoflinks):
      print('T' + str(i+1) + ' = ' + str(T[i]))

# Preparing arrays of terms to accept valid input
q = []
qdot = []
qddot = []
for i in range(1000):
  q.append(sp.symbols('q' + str(i + 1)))
  qdot.append(sp.symbols('q"' + str(i + 1)))
  qddot.append(sp.symbols('q""' + str(i + 1)))

# Initializing object robot with input [numberoflinks, D_q, V_q]
robot = Robot(2, [[5, 15 * sp.cos(q[1] - q[0])], [15 * sp.cos(q[0] - q[0]), 10]], 13 * sp.sin(q[0]) + 45 * (5 * sp.sin(q[0]) + 3 * sp.sin(q[0])))
robot.derive_equations()

T1 = 5*q""1 + 15*q""2*cos(q1 - q2) + 15.0*q"1**2*sin(q1 - q2) + 30.0*q"1*q"2*sin(q1 - q2) + 15.0*q"2**2*sin(q1 - q2) + 373*cos(q1)
T2 = 20*q""1 + 15*q""2*cos(q1 - q2) + 10*q""2 + 15.0*q"1**2*sin(q1 - q2) + 30.0*q"1*q"2*sin(q1 - q2) + 15.0*q"2**2*sin(q1 - q2)
